In [1]:
import pymc4 as pm
import tensorflow as tf
import numpy as np

## Graph

In [9]:
sess = tf.InteractiveSession()

In [3]:
@pm.model
def t_test(sd_prior='half_normal'):
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd', 1)
    pm.Normal('y_0', 0, 2 * sd)
    pm.Normal('y_1', mu, 2 * sd)

model = t_test.configure()

model._forward_context.vars

In [76]:
mu = tf.placeholder(tf.float32)
sd = tf.placeholder(tf.float32)
y_0 = tf.placeholder(tf.float32)
y_1 = tf.placeholder(tf.float32)
logp = func(mu, sd, y_0, y_1)

mu_ = np.ones(10).astype('f')
sd_ = np.ones(10).astype('f')
y_0_ = np.ones(10).astype('f')
y_1_ = np.ones(10).astype('f')
%timeit sess.run(logp, feed_dict={mu: mu_, sd: sd_, y_0: y_0_, y_1: y_1_})
sess.run(logp, feed_dict={mu: mu_, sd: sd_, y_0: y_0_, y_1: y_1_})

508 µs ± 32.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


-54.939014

In [77]:
grad = tf.gradients(logp, [mu, sd, y_0, y_1])
%timeit sess.run([logp, grad], feed_dict={mu: mu_, sd: sd_, y_0: y_0_, y_1: y_1_})
sess.run([logp, grad], feed_dict={mu: mu_, sd: sd_, y_0: y_0_, y_1: y_1_})

2.14 ms ± 322 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


[-54.939014,
 [array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.], dtype=float32),
  array([-2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75,
         -2.75], dtype=float32),
  array([-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25,
         -0.25], dtype=float32),
  array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.], dtype=float32)]]

In [78]:
array = tf.placeholder(tf.float32)

mu = array[:10]
sd = array[10:20]
y_0 = array[20:30]
y_1 = array[30:40]
logp = func(mu, sd, y_0, y_1)

array_ = np.ones(40).astype('f')
%timeit sess.run(logp, {array: array_})
sess.run(logp, {array: array_})

424 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


-54.939014

In [79]:
grad = tf.gradients(logp, array)
%timeit sess.run([logp, grad], {array: array_})
sess.run([logp, grad], {array: array_})

2.26 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


[-54.939014,
 [array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,
         -1.  , -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75,
         -2.75, -2.75, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25,
         -0.25, -0.25, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
          0.  ,  0.  ,  0.  ,  0.  ], dtype=float32)]]

In [84]:
mu = tf.Variable(tf.ones(10), dtype=tf.float32)[:]
sd = tf.Variable(tf.ones(10), dtype=tf.float32)[:]
y_0 = tf.Variable(tf.ones(10), dtype=tf.float32)[:]
y_1 = tf.Variable(tf.ones(10), dtype=tf.float32)[:]
logp = func(mu, sd, y_0, y_1)

sess.run(tf.global_variables_initializer())
%timeit sess.run(logp)
sess.run(logp)

336 µs ± 21 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


-54.939014

In [86]:
array = tf.Variable(tf.ones(40), dtype=tf.float32)
grad_array = tf.Variable(tf.zeros([1, 40]), dtype=tf.float32)

mu = array[:10]
sd = array[10:20]
y_0 = array[20:30]
y_1 = array[30:40]
logp = func(mu, sd, y_0, y_1)
grad = tf.gradients(logp, array)
write = grad_array.assign(grad, read_value=False)

sess.run(tf.global_variables_initializer())

%timeit sess.run([logp, write])
sess.run([logp, write])

1.88 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


[-54.939014, None]

In [90]:
%timeit sess.run(grad_array)
sess.run(grad_array)

79.9 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


array([[-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,
        -1.  , -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75,
        -2.75, -2.75, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25,
        -0.25, -0.25, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ]], dtype=float32)

## Eager

In [6]:
tf.enable_eager_execution()

In [7]:
@pm.model
def t_test(sd_prior='half_normal'):
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd', 1)
    pm.Normal('y_0', 0, 2 * sd)
    pm.Normal('y_1', mu, 2 * sd)

model = t_test.configure()

model._forward_context.vars

In [15]:
func = model.make_logp_function()

mu = tf.ones((10,))
sd = tf.ones((10,))
y_0 = tf.ones((10,))
y_1 = tf.ones((10,))
%timeit logp = func(mu, sd, y_0, y_1)
func(mu, sd, y_0, y_1)

897 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


<tf.Tensor: id=1800979, shape=(), dtype=float32, numpy=-54.939014>

In [11]:
logp_func_defun = tf.contrib.eager.defun(func)

In [14]:
mu = tf.ones((10,))
sd = tf.ones((10,))
y_0 = tf.ones((10,))
y_1 = tf.ones((10,))
%timeit logp = logp_func_defun(mu, sd, y_0, y_1)
logp_func_defun(mu, sd, y_0, y_1)

104 µs ± 3.59 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<tf.Tensor: id=1281798, shape=(), dtype=float32, numpy=-54.939014>

In [21]:
mu = tf.ones((10,))
sd = tf.ones((10,))
y_0 = tf.ones((10,))
y_1 = tf.ones((10,))

with tf.GradientTape() as tape:
    tape.watch(mu)
    tape.watch(sd)
    tape.watch(y_0)
    tape.watch(y_1)

    logp = logp_func_defun(mu, sd, y_0, y_1)

tape.gradient(logp, mu)

<tf.Tensor: id=1882584, shape=(10,), dtype=float32, numpy=array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.], dtype=float32)>

In [24]:
%%timeit
with tf.GradientTape() as tape:
    tape.watch(mu)
    tape.watch(sd)
    tape.watch(y_0)
    tape.watch(y_1)
    logp = logp_func_defun(mu, sd, y_0, y_1)

tape.gradient(logp, [mu, sd, y_0, y_1])

883 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
def logp_and_grad(*args):
    logp = func(*args)
    return logp, tf.gradients(logp, args)

logp_grad_func_defun = tf.contrib.eager.defun(logp_and_grad)

mu = tf.ones((10,))
sd = tf.ones((10,))
y_0 = tf.ones((10,))
y_1 = tf.ones((10,))
%timeit logp = logp_grad_func_defun(mu, sd, y_0, y_1)
logp_grad_func_defun(mu, sd, y_0, y_1)

1.58 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


(<tf.Tensor: id=2516112, shape=(), dtype=float32, numpy=-54.939014>,
 [<tf.Tensor: id=2516113, shape=(10,), dtype=float32, numpy=array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.], dtype=float32)>,
  <tf.Tensor: id=2516114, shape=(10,), dtype=float32, numpy=
  array([-2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75, -2.75,
         -2.75], dtype=float32)>,
  <tf.Tensor: id=2516115, shape=(10,), dtype=float32, numpy=
  array([-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25,
         -0.25], dtype=float32)>,
  <tf.Tensor: id=2516116, shape=(10,), dtype=float32, numpy=array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.], dtype=float32)>])